What is Elliptic Curve Cryptography:
1. Elliptic Curve (ECs): $(y^2 = x^3 + ax + b)
2. Finite Fields are used for computations on the curve point 
3. Base point: starting point on the curve
4. Private and Public Keys Crypto with ECC
* Private key is random, public key is on the curve derived from the private key 


Security Basis: 
* Given two points P and Q on the curve it is infeasible to find a scalar where Q=k*p

Real World Basis: 
* The Sony ECDSA security disaster refers to a critical vulnerability in the implementation of the Elliptic Curve Digital Signature Algorithm (ECDSA) on the PlayStation 3 (PS3) gaming console. This flaw severely compromised the console's security and led to significant consequences for Sony and the gaming community.


In [39]:
import ecdsa
import os

def generate_keypair():
    # Using a weak curve (NIST192p) instead of a stronger one
    return ecdsa.SigningKey.generate(curve=ecdsa.NIST192p)

def sign_message(private_key, message):
    fixed_nonce = b'1234567890'
    return private_key.sign(message.encode(), k=int.from_bytes(fixed_nonce, 'big'))

def verify_signature(public_key, message, signature):
    try:
        return public_key.verify(signature, message.encode())
    except:
        return False

# Example usage
private_key = generate_keypair()
public_key = private_key.get_verifying_key()

message = "This is a secret message"
signature = sign_message(private_key, message)

print(f"Signature: {signature.hex()}")
print(f"Verification result: {verify_signature(public_key, message, signature)}")

Signature: 68384b6279e49026e39704b8a08c04a2c4384ecabd73ad194c5cd8298dd2f49f7b2e3fd60738ab6a0016a29906cf9f95
Verification result: True




The ECDSA (Elliptic Curve Digital Signature Algorithm) works based on the mathematics of elliptic curves over finite fields:

## Elliptic Curve Equation

The fundamental equation for ECDSA is:

y^2 = (x^3 + ax + b) mod p

Where:
- a and b are curve parameters
- p is a large prime number (the field size)

## Key Generation

1. Private key: A random integer d in the range [1, n-1], where n is the order of the curve.
2. Public key: Q = dG, where G is the generator point of the curve.

## Signing Process

To sign a message m:

1. Calculate e = HASH(m)
2. Generate a random nonce k in the range [1, n-1]
3. Calculate the curve point (x_1, y_1) = kG
4. Calculate r = x_1 mod n
5. Calculate s = k^(-1)(e + dr) mod n

The signature is the pair (r, s).

## Verification Process

To verify a signature (r, s) for a message m:

1. Calculate e = HASH(m)
2. Calculate w = s^(-1) mod n
3. Calculate u_1 = ew mod n and u_2 = rw mod n
4. Calculate the curve point (x, y) = u_1G + u_2Q
5. The signature is valid if r ≡ x mod n

## Security Principle

The security of ECDSA relies on the difficulty of the Elliptic Curve Discrete Logarithm Problem (ECDLP). Given points P and Q on the curve, it's computationally infeasible to find a scalar k such that Q = kP when the curve is sufficiently large.


In [47]:
import ecdsa
import os

def generate_keypair():
    return ecdsa.SigningKey.generate(curve=ecdsa.NIST192p)

def sign_message(private_key, message):
    fixed_nonce = b'1234567890'
    return private_key.sign(message.encode(), k=int.from_bytes(fixed_nonce, 'big'))

def verify_signature(public_key, message, signature):
    try:
        return public_key.verify(signature, message.encode())
    except:
        return False

# Generate keys and sign a message
private_key = generate_keypair()
public_key = private_key.get_verifying_key()

message1 = "This is message 1"
signature1 = sign_message(private_key, message1)

message2 = "This is message 2"
signature2 = sign_message(private_key, message2)

print(f"Signature 1: {signature1.hex()}")
print(f"Signature 2: {signature2.hex()}")

Signature 1: 68384b6279e49026e39704b8a08c04a2c4384ecabd73ad19da3eaf8e9e7a52c73f7d7cae18684d360db495f1677e5087
Signature 2: 68384b6279e49026e39704b8a08c04a2c4384ecabd73ad19b5ad8fe07fa10030b18fc1b07feb75b83b6a32bee0da0308




s =K^-1(z + xr) mod n

where: 
* k is nonce
* z is the hash
* x is the private key 
* n is the order of the curve 

s_1- s_2= k^(-1)(z_1-z_2) mod n 

k = (z1 - z2) * (s1 - s2)^9(-1) mod n 

In [48]:
import hashlib

def forge_signature(public_key, message1, signature1, message2, signature2):
    # Convert signatures to integers
    r, s1 = ecdsa.util.sigdecode_string(signature1, public_key.curve.order)
    _, s2 = ecdsa.util.sigdecode_string(signature2, public_key.curve.order)
    
    # Calculate the difference of message hashes
    z1 = int.from_bytes(hashlib.sha1(message1.encode()).digest(), 'big')
    z2 = int.from_bytes(hashlib.sha1(message2.encode()).digest(), 'big')
    
     # Recover the nonce (k)
    k = ((z1 - z2) * pow(s1 - s2, -1, public_key.pubkey.order)) % public_key.pubkey.order
    
    # Recover the private key
    private_key = ((s1 * k - z1) * pow(r, -1, public_key.pubkey.order)) % public_key.pubkey.order
    
    # Create a new signing key from the recovered private key
    recovered_key = ecdsa.SigningKey.from_secret_exponent(private_key, curve=public_key.curve)
    
    return recovered_key


# Forge a signature for any message
forged_key = forge_signature(public_key, message1, signature1, message2, signature2)
forged_message = "admin"
forged_signature = sign_message(forged_key, forged_message)

print(f"Forged message: {forged_message}")
print(f"Forged signature: {forged_signature.hex()}")
print(f"Verification of forged signature: {verify_signature(public_key, forged_message, forged_signature)}")

Forged message: admin
Forged signature: 68384b6279e49026e39704b8a08c04a2c4384ecabd73ad19bafc68f5ddfa0333bde5817ea671f782086f27072039f870
Verification of forged signature: True
